<a href="https://colab.research.google.com/github/Vikas-Soni-nn/Test/blob/Vikas/Interpenetration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install libboost-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-dev is already the newest version (1.65.1.0ubuntu1).
libboost-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [ ]:
!pip install git+https://github.com/MPI-IS/mesh.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MPI-IS/mesh.git to /tmp/pip-req-build-utmaaymj
  Running command git clone -q https://github.com/MPI-IS/mesh.git /tmp/pip-req-build-utmaaymj
  Created wheel for psbody-mesh: filename=psbody_mesh-0.4-cp37-cp37m-linux_x86_64.whl size=1855236 sha256=20d58140b5e9f4517fa05df4040f427a6a2064f8120924e2d28e0b5ec875887d
  Stored in directory: /tmp/pip-ephem-wheel-cache-i1f4fvy_/wheels/15/9e/8a/9a8acd7eb649a805222ec1acd1dad2abb02a333d74fb5c6afc
Successfully built psbody-mesh


In [ ]:
!git clone https://github.com/chaitanya100100/TailorNet.git

Cloning into 'TailorNet'...
remote: Enumerating objects: 327, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 327 (delta 81), reused 71 (delta 71), pack-reused 236
Receiving objects: 100% (327/327), 7.13 MiB | 12.78 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [ ]:
cd TailorNet


/content/TailorNet


In [ ]:
import numpy as np

import scipy.sparse as sp
from scipy.sparse import vstack, csr_matrix
from scipy.sparse.linalg import spsolve   # ILNumerics
from psbody.mesh import Mesh

from psbody.mesh import Mesh
from psbody.mesh.geometry.vert_normals import VertNormals
from psbody.mesh.geometry.tri_normals import TriNormals
from psbody.mesh.search import AabbTree
from utils.diffusion_smoothing import numpy_laplacian_uniform as laplacian

import time 
def get_nearest_points_and_normals(vert, base_verts, base_faces):
    """For each vertex of `vert`, find nearest surface points on
    base mesh (`base_verts`, `base_faces`).
    """
    fn = TriNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    #print("fn", fn)
    vn = VertNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    # np.save('/content/vertex_normals.npy', vn)
    # np.save('/content/faces_normals.npy', fn)
    #print("vn", vn)
    #start_time = time.time()
    tree = AabbTree(Mesh(v=base_verts, f=base_faces))
    #print(time.time()- start_time)
    nearest_tri, nearest_part, nearest_point = tree.nearest(vert, nearest_part=True)
    # np.save('/content/nearest_tri.npy', nearest_tri)
    # np.save('/content/nearest_part.npy', nearest_part)
    # np.save('/content/nearest_point.npy', nearest_point)
    #print(time.time() - start_time)
    nearest_tri = nearest_tri.ravel().astype(np.long)
    nearest_part = nearest_part.ravel().astype(np.long)

    nearest_normals = np.zeros_like(vert)

    # nearest_part tells you whether the closest point in triangle abc is
    # in the interior (0), on an edge (ab:1,bc:2,ca:3), or a vertex (a:4,b:5,c:6)
    cl_tri_idxs = np.nonzero(nearest_part == 0)[0].astype(np.int)
    cl_vrt_idxs = np.nonzero(nearest_part > 3)[0].astype(np.int)
    cl_edg_idxs = np.nonzero((nearest_part <= 3) & (nearest_part > 0))[0].astype(np.int)
    print(cl_tri_idxs.shape)
    nt = nearest_tri[cl_tri_idxs]
    nearest_normals[cl_tri_idxs] = fn[nt]

    nt = nearest_tri[cl_vrt_idxs]
    npp = nearest_part[cl_vrt_idxs] - 4
    nearest_normals[cl_vrt_idxs] = vn[base_faces[nt, npp]]

    nt = nearest_tri[cl_edg_idxs]
    npp = nearest_part[cl_edg_idxs] - 1
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]
    npp = np.mod(nearest_part[cl_edg_idxs], 3)
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]

    nearest_normals = nearest_normals / (np.linalg.norm(nearest_normals, axis=-1, keepdims=True) + 1.e-10)

    return nearest_point, nearest_normals


def remove_interpenetration_fast(mesh, base, L=None):
    """Deforms `mesh` to remove its interpenetration from `base`.
    This is posed as least square optimization problem which can be solved
    faster with sparse solver.
    """

    eps = 0.01
    ww = 2.0
    nverts = mesh.v.shape[0]

    if L is None:
        L = laplacian(mesh.v, mesh.f)

    nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, base.v, base.f)
    direction = np.sign( np.sum((mesh.v - nearest_points) * nearest_normals, axis=-1) )

    indices = np.where(direction < 0)[0]

    pentgt_points = nearest_points[indices] - mesh.v[indices]
    pentgt_points = [indices] + eps * pentgt_points / np.expand_dims(0.0001 + np.linalg.norm(pentgt_points, axis=1), 1)
    tgt_points = mesh.v.copy()
    tgt_points[indices] = ww * pentgt_points

    rc = np.arange(nverts)
    data = np.ones(nverts)
    data[indices] *= ww
    #np.save('/content/data.npy', data)
    I = csr_matrix((data, (rc, rc)), shape=(nverts, nverts))
    #print(I.shape)

    A = vstack([L, I])
    b = np.vstack((
        L.dot(mesh.v),
        tgt_points
    ))
    #np.save('/content/matrix.npy', I)
    start_time = time.time()
    res = spsolve(A.T.dot(A), A.T.dot(b))
    print(time.time() - start_time)
    mres = Mesh(v=res, f=mesh.f)
    return mres




In [ ]:
import os
dir = "/content/drive/MyDrive/1_10_anim_body"
dir_cloth = '/content/drive/MyDrive/anim_out'
for i in range(370):
    filename = f'anim_body_{i}.obj'
    f = os.path.join(dir, filename)
    filename_cloth = f'anim_cloth_{i}.obj'
    f_cloth = os.path.join(dir_cloth, filename_cloth)
    body = Mesh(filename=f)
    cloth = Mesh(filename=f_cloth)
    tree = AabbTree(Mesh(v=body.v, f=body.f))
    nearest_tri, nearest_part, nearest_point = tree.nearest(cloth.v, nearest_part=True)
    #fn = TriNormals(v=body.v, f=body.f).reshape((-1, 3))
    np.save(f'/content/drive/MyDrive/nearest_tri/nearest_tri_{i}.npy', nearest_tri)
    np.save(f'/content/drive/MyDrive/nearest_point/nearest_point_{i}.npy', nearest_point)

# body = Mesh(filename='/content/drive/MyDrive/1.obj')
# mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
# fn = TriNormals(v=base_verts, f=base_faces).reshape((-1, 3))

# nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)

In [ ]:
body = Mesh(filename='/content/drive/MyDrive/1_000001.obj')
mesh = Mesh(filename='/content/drive/MyDrive/anim_out/anim_cloth_0.obj')
import time
start_time = time.time()
tree = AabbTree(Mesh(v=body.v, f=body.f))
print(tree)
nearest_tri, nearest_part, nearest_point = tree.nearest(mesh.v, nearest_part=True)
print(time.time() - start_time)

0.052698612213134766


In [ ]:
nearest_tri

array([[ 8726,  8722,  5465, ..., 13664, 18163, 13793]], dtype=uint32)

In [ ]:
np.save('/content/nearest_normals.npy', nearest_normals)

In [ ]:
direction = np.sign( np.sum((mesh.v - nearest_points) * nearest_normals, axis=-1) )

indices = np.where(direction < 0)[0]

In [ ]:
indices
np.save('/content/points-Inside.npy', indices)

In [ ]:
body = Mesh(filename='/content/drive/MyDrive/1_10_anim_body/anim_body_225.obj')
mesh = Mesh(filename='/content/drive/MyDrive/anim_out/anim_cloth_225.obj')
print(body.v.shape)
mesh.v.shape

(10475, 3)


(6716, 3)

In [ ]:
import time
#start_time_int = time.time()
body = Mesh(filename='/content/drive/MyDrive/1_10_anim_body/anim_body_225.obj')
mesh = Mesh(filename='/content/drive/MyDrive/anim_out/anim_cloth_225.obj')
start_time_int = time.time()
mesh1 = remove_interpenetration_fast(mesh, body)

mesh1.write_obj('/content/gar_naive_2.obj')

#end_time = time.time()
print(time.time() - start_time_int) 

(5384,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

ValueError: ignored

In [ ]:
from scipy.spatial import KDTree
body = Mesh(filename='/content/drive/MyDrive/1.obj')
distances = [0]*6716
indices = [0]*6716
import time
start_time = time.time()


kdtree = KDTree(body.v)
print(time.time() - start_time)
mesh_cloth = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
vertices = mesh_cloth.v
# distances, indices = kdtree.query(vertices)
# print(time.time() - start_time)
for i in range(6716):
  distances[i], indices[i] = kdtree.query(vertices[i])
print(time.time() - start_time)


0.004909992218017578
0.22987031936645508


In [ ]:
a = []
for i in range(len(indices)):
        b = [indices[i], indices[i] + 1, indices[i]+2, indices[i]+3, indices[i]+4, indices[i]+5, indices[i]+6, indices[i]+7, indices[i]+8,indices[i]+9]
        a.append(b)
        

In [ ]:
a[1]

[8233, 8234, 8235, 8236, 8237, 8238, 8239, 8240, 8241, 8242]

In [ ]:
np.save('/content/vertex2vertex.npy', a)


In [ ]:
for i in range(len())


## Naive interpenetration

In [ ]:
import numpy as np

import scipy.sparse as sp
from scipy.sparse import vstack, csr_matrix
from scipy.sparse.linalg import spsolve   # ILNumerics
from psbody.mesh import Mesh

from psbody.mesh import Mesh
from psbody.mesh.geometry.vert_normals import VertNormals
from psbody.mesh.geometry.tri_normals import TriNormals
from psbody.mesh.search import AabbTree
from utils.diffusion_smoothing import numpy_laplacian_uniform as laplacian

import time 
def get_nearest_points_and_normals(vert, base_verts, base_faces):
    """For each vertex of `vert`, find nearest surface points on
    base mesh (`base_verts`, `base_faces`).
    """
    fn = TriNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    #print("fn", fn)
    vn = VertNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    #print("vn", vn)
    #start_time = time.time()
    tree = AabbTree(Mesh(v=base_verts, f=base_faces))
    
    #print(time.time()- start_time)
    nearest_tri, nearest_part, nearest_point = tree.nearest(vert, nearest_part=True)
    #print(nearest_point)
    #print(nearest_tri)
    #print(nearest_tri.shape)
    #print(time.time() - start_time)
    nearest_tri = nearest_tri.ravel().astype(np.long)
    nearest_part = nearest_part.ravel().astype(np.long)

    nearest_normals = np.zeros_like(vert)

    # nearest_part tells you whether the closest point in triangle abc is
    # in the interior (0), on an edge (ab:1,bc:2,ca:3), or a vertex (a:4,b:5,c:6)
    cl_tri_idxs = np.nonzero(nearest_part == 0)[0].astype(np.int)
    print(cl_tri_idxs)
    print(cl_tri_idxs.shape)
    cl_vrt_idxs = np.nonzero(nearest_part > 3)[0].astype(np.int)
    print(cl_vrt_idxs)
    print(cl_vrt_idxs.shape)
    cl_edg_idxs = np.nonzero((nearest_part <= 3) & (nearest_part > 0))[0].astype(np.int)
    print(cl_edg_idxs)
    print(cl_edg_idxs.shape)

    nt = nearest_tri[cl_tri_idxs]
    print(nt)
    nearest_normals[cl_tri_idxs] = fn[nt]

    nt = nearest_tri[cl_vrt_idxs]
    npp = nearest_part[cl_vrt_idxs] - 4
    nearest_normals[cl_vrt_idxs] = vn[base_faces[nt, npp]]

    nt = nearest_tri[cl_edg_idxs]
    npp = nearest_part[cl_edg_idxs] - 1
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]
    npp = np.mod(nearest_part[cl_edg_idxs], 3)
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]

    nearest_normals = nearest_normals / (np.linalg.norm(nearest_normals, axis=-1, keepdims=True) + 1.e-10)

    return nearest_point, nearest_normals


def remove_interpenetration_fast(mesh, base, L=None):
    """Deforms `mesh` to remove its interpenetration from `base`.
    This is posed as least square optimization problem which can be solved
    faster with sparse solver.
    """

    eps = 0.01
    ww = 2.0
    nverts = mesh.v.shape[0]

    if L is None:
        L = laplacian(mesh.v, mesh.f)

    nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, base.v, base.f)
    direction = np.sign( np.sum((mesh.v - nearest_points) * nearest_normals, axis=-1) )

    indices = np.where(direction < 0)[0]
    print(indices)
    for index in indices:
      mesh.v[index] = nearest_points[index]+ 0.1
    mres = Mesh(v=mesh.v, f=mesh.f)

    # pentgt_points = nearest_points[indices] - mesh.v[indices]
    # pentgt_points = nearest_points[indices] \
    #                 + eps * pentgt_points / np.expand_dims(0.0001 + np.linalg.norm(pentgt_points, axis=1), 1)
    # tgt_points = mesh.v.copy()
    # tgt_points[indices] = ww * pentgt_points

    # rc = np.arange(nverts)
    # data = np.ones(nverts)
    # data[indices] *= ww
    # #np.save('/content/data.npy', data)
    # I = csr_matrix((data, (rc, rc)), shape=(nverts, nverts))
    # #print(I.shape)

    # A = vstack([L, I])
    # b = np.vstack((
    #     L.dot(mesh.v),
    #     tgt_points
    # ))
    # #np.save('/content/matrix.npy', I)
    # start_time = time.time()
    # res = spsolve(A.T.dot(A), A.T.dot(b))
    # print(time.time() - start_time)
    # mres = Mesh(v=res, f=mesh.f)
    return mres

In [ ]:
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)


[   0    1    3 ... 6713 6714 6715]
(5571,)
[ 335  376  441  791  823  838 1045 1892 2869 2950 3155 3157 3159 3206
 3772 4049 4267 4419 4424 4437 4574 4580 4583 4834 4922 5560 6317 6448
 6461 6469 6514 6515 6537 6538 6641 6658]
(36,)
[   2   20   32 ... 6709 6710 6711]
(1109,)
[ 8726  8722  8729 ... 13664 18163 13793]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
nearest_points[0]


array([-0.15373073, -3.28210668,  1.45027695])

## Nearest points

In [ ]:
import numpy as np

import scipy.sparse as sp
from scipy.sparse import vstack, csr_matrix
from scipy.sparse.linalg import spsolve   # ILNumerics
from psbody.mesh import Mesh

from psbody.mesh import Mesh
from psbody.mesh.geometry.vert_normals import VertNormals
from psbody.mesh.geometry.tri_normals import TriNormals
from psbody.mesh.search import AabbTree
from utils.diffusion_smoothing import numpy_laplacian_uniform as laplacian


import time 
def get_nearest_points_and_normals(vert, base_verts, base_faces):
    """For each vertex of `vert`, find nearest surface points on
    base mesh (`base_verts`, `base_faces`).
    """
    fn = TriNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    #print("fn", fn)
    
    #print("vn", vn)vn = VertNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    #start_time = time.time()
    tree = AabbTree(Mesh(v=base_verts, f=base_faces))
    #print(time.time()- start_time)
    nearest_tri, nearest_part, nearest_point, nearest_normal = tree.nearest(vert, nearest_part=True)
    print(nearest_point)
    #print(time.time() - start_time)
    nearest_tri = nearest_tri.ravel().astype(np.long)
    nearest_part = nearest_part.ravel().astype(np.long)

    #nearest_normals = 

    nearest_normals = np.zeros_like(vert)

    # nearest_part tells you whether the closest point in triangle abc is
    # in the interior (0), on an edge (ab:1,bc:2,ca:3), or a vertex (a:4,b:5,c:6)
    cl_tri_idxs = np.nonzero(nearest_part == 0)[0].astype(np.int)
    cl_vrt_idxs = np.nonzero(nearest_part > 3)[0].astype(np.int)
    cl_edg_idxs = np.nonzero((nearest_part <= 3) & (nearest_part > 0))[0].astype(np.int)

    nt = nearest_tri[cl_tri_idxs]
    nearest_normals[cl_tri_idxs] = fn[nt]

    nt = nearest_tri[cl_vrt_idxs]
    npp = nearest_part[cl_vrt_idxs] - 4
    nearest_normals[cl_vrt_idxs] = vn[base_faces[nt, npp]]

    nt = nearest_tri[cl_edg_idxs]
    npp = nearest_part[cl_edg_idxs] - 1
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]
    npp = np.mod(nearest_part[cl_edg_idxs], 3)
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]

    nearest_normals = nearest_normals / (np.linalg.norm(nearest_normals, axis=-1, keepdims=True) + 1.e-10)

    return nearest_point, nearest_normals


def remove_interpenetration_fast(mesh, base, L=None):
    """Deforms `mesh` to remove its interpenetration from `base`.
    This is posed as least square optimization problem which can be solved
    faster with sparse solver.
    """

    eps = 0.01
    ww = 2.0
    nverts = mesh.v.shape[0]

    if L is None:
        L = laplacian(mesh.v, mesh.f)

    nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, base.v, base.f)
    direction = np.sign( np.sum((mesh.v - nearest_points) * nearest_normals, axis=-1) )

    indices = np.where(direction < 0)[0]
    print(indices)
    for index in indices:
      mesh.v[index] = nearest_points[index]+ 0.1
      #main
    

    mres = Mesh(v=mesh.v, f=mesh.f)

    # pentgt_points = nearest_points[indices] - mesh.v[indices]
    # pentgt_points = nearest_points[indices] \
    #                 + eps * pentgt_points / np.expand_dims(0.0001 + np.linalg.norm(pentgt_points, axis=1), 1)
    # tgt_points = mesh.v.copy()
    # tgt_points[indices] = ww * pentgt_points

    # rc = np.arange(nverts)
    # data = np.ones(nverts)
    # data[indices] *= ww
    # #np.save('/content/data.npy', data)
    # I = csr_matrix((data, (rc, rc)), shape=(nverts, nverts))
    # #print(I.shape)

    # A = vstack([L, I])
    # b = np.vstack((
    #     L.dot(mesh.v),
    #     tgt_points
    # ))
    # #np.save('/content/matrix.npy', I)
    # start_time = time.time()
    # res = spsolve(A.T.dot(A), A.T.dot(b))
    # print(time.time() - start_time)
    # mres = Mesh(v=res, f=mesh.f)
    return mres
  


In [ ]:
nearest_indices = np.load('/content/nearest_indices.npy')



In [ ]:
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)

[   0    1    3 ... 6713 6714 6715]
(5571,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:

body = Mesh(filename='/content/drive/MyDrive/1_000001.obj')
mesh = Mesh(filename='/content/drive/MyDrive/anim_out/anim_cloth_0.obj')
import time
start_time = time.time()
tree = AabbTree(Mesh(v=body.v, f=body.f))
print(tree)
nearest_tri, nearest_part, nearest_point = tree.nearest(mesh.v, nearest_part=True)
print(time.time() - start_time)

0.057584524154663086


In [ ]:
nearest_tri
np.save('/content/nearest_tri.npy', nearest_tri)

In [ ]:
1/0.04

25.0

In [ ]:
import time 
start_time = time.time()
L = laplacian(mesh.v, mesh.f)
print(time.time() - start_time)
body.v.shape


0.017035484313964844


(10475, 3)

In [ ]:
import time
#start_time_int = time.time()
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
start_time_int = time.time()
mesh1 = remove_interpenetration_fast(mesh, body)

mesh1.write_obj('/content/gar_naive_2.obj')

#end_time = time.time()
print(time.time() - start_time_int)

[   0    1    3 ... 6713 6714 6715]
(5571,)
[ 335  376  441  791  823  838 1045 1892 2869 2950 3155 3157 3159 3206
 3772 4049 4267 4419 4424 4437 4574 4580 4583 4834 4922 5560 6317 6448
 6461 6469 6514 6515 6537 6538 6641 6658]
(36,)
[   2   20   32 ... 6709 6710 6711]
(1109,)
[ 8726  8722  8729 ... 13664 18163 13793]
[  59   77  144  145  147  148 1362 1365 1922 2656 2657 2665 2666 2674
 2675 2750 2756 2757 2761 2762 2767 3467 3468 3472 3531 3532 3533 3534
 3540 4720 4753 5992 6075 6079 6080 6081 6083 6084 6086 6090]
0.10836410522460938


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
import time
#start_time_int = time.time()
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
start_time_int = time.time()
mesh1 = remove_interpenetration_fast(mesh, body)

mesh1.write_obj('/content/gar_naive_2.obj')

#end_time = time.time()
print(time.time() - start_time_int)

[  59   77  144  145  147  148 1362 1365 1922 2656 2657 2665 2666 2674
 2675 2750 2756 2757 2761 2762 2767 3467 3468 3472 3531 3532 3533 3534
 3540 4720 4753 5992 6075 6079 6080 6081 6083 6084 6086 6090]
0.24290919303894043


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
L

<6716x6716 sparse matrix of type '<class 'numpy.float64'>'
	with 46488 stored elements in Compressed Sparse Row format>

In [ ]:
def numpy_laplacian_uniform(v, f):
    """Computes uniform laplacian operator on mesh."""
    import scipy.sparse as sp
    from sklearn.preprocessing import normalize
    from psbody.mesh.topology.connectivity import get_vert_connectivity

    connectivity = get_vert_connectivity(Mesh(v=v, f=f))
    # connectivity is a sparse matrix, and np.clip can not applied directly on
    # a sparse matrix.
    connectivity.data = np.clip(connectivity.data, 0, 1)
    lap = normalize(connectivity, norm='l1', axis=1)
    lap = lap - sp.eye(connectivity.shape[0])

    return lap

## CSR matrix, vertical stack, sparsesolver for least square optimization problem


In [ ]:


tree = AabbTree(Mesh(v=base_verts, f=base_faces))


In [ ]:
w = 2, ep = 0.001

In [ ]:
from scipy.spatial import KDTree
body = Mesh(filename='/content/drive/MyDrive/1.obj')
distances = [0]*6716
indices = [0]*6716
import time
start_time = time.time()


kdtree = KDTree(body.v)
print(time.time() - start_time)
mesh_cloth = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
vertices = mesh_cloth.v
# distances, indices = kdtree.query(vertices)
# print(time.time() - start_time)
for i in range(6716):
  distances[i], indices[i] = kdtree.query(vertices[i])
print(time.time() - start_time)


0.0070209503173828125
0.4160747528076172


In [ ]:
indices
np.save('/content/nearest_indices.npy', indices)

In [ ]:
0.26/0.05

5.2

In [ ]:
1/0.05

20.0

In [ ]:
distances

array([0.24547372, 0.23997595, 0.23243552, ..., 0.46294087, 0.14164311,
       0.12230339])

In [ ]:
a = np.load('/content/data.npy',allow_pickle=True)
b = 0
for i in range(6716):
  if a[i] == 1:
    b = b+1 
b

6676

In [ ]:
6716 - 6676

40

In [ ]:
import time
#start_time = time.time()
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
nearest_indices = np.load('nearest_indices.npy')
for i in range(6716):
    

In [ ]:
import time
#start_time = time.time()
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
start_time = time.time()
mesh1 = remove_interpenetration_fast(mesh, body, L)

#mesh1.write_obj('/content/gar_20_.05.obj')

#end_time = time.time()
print(time.time() - start_time)

# print(mesh.v.shape[0])
# rc = np.arange(mesh.v.shape[0])
# print(rc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

0.4241940975189209


In [ ]:
1/0.013433218002319336

74.44232646468949

In [ ]:
mesh1.v

array([[-1.73853082e-01,  9.09436115e-01, -8.91082190e-03],
       [-9.54339517e-04,  9.00409649e-01,  9.73733325e-02],
       [-1.93782364e-01,  1.42188015e+00, -4.43484621e-02],
       ...,
       [ 3.46702998e-02,  1.43821499e+00, -7.91721148e-02],
       [ 1.26020829e-02,  1.43801687e+00, -8.20921669e-02],
       [ 5.70707052e-02,  1.43959635e+00, -7.53496541e-02]])

In [ ]:
start_time = time.time()
mesh1 = remove_interpenetration_fast(mesh, body)
end_time = time.time()
print(end_time - start_time)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

0.3804140090942383


In [ ]:
start_time = time.time()
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)
end_time = time.time()
print(end_time - start_time)

0.035628318786621094
0.07294130325317383


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
1/0.035


28.57142857142857

In [ ]:
%timeit nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

10 loops, best of 5: 59.9 ms per loop


In [ ]:
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)
print(mesh.v.shape)
nearest_points.shape

(6716, 3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

(6716, 3)

In [ ]:

import sys
#sys.path.append('../MGN_release')
import numpy as np

from psbody.mesh import Mesh
from psbody.mesh.geometry.vert_normals import VertNormals
from psbody.mesh.geometry.tri_normals import TriNormals
from psbody.mesh.search import AabbTree


def laplacian(part_mesh):
    """ Compute laplacian operator on part_mesh. This can be cached.
    """
    import scipy.sparse as sp
    from sklearn.preprocessing import normalize
    from psbody.mesh.topology.connectivity import get_vert_connectivity

    connectivity = get_vert_connectivity(part_mesh)
    # connectivity is a sparse matrix, and np.clip can not applied directly on
    # a sparse matrix.
    connectivity.data = np.clip(connectivity.data, 0, 1)
    lap = normalize(connectivity, norm='l1', axis=1)
    lap = sp.eye(connectivity.shape[0]) - lap

    return lap


# inspired from frankengeist.body.ch.mesh_distance.MeshDistanceSquared
def get_nearest_points_and_normals(vert, base_verts, base_faces):

    fn = TriNormals(v=base_verts, f=base_faces).reshape((-1, 3))
    vn = VertNormals(v=base_verts, f=base_faces).reshape((-1, 3))

    tree = AabbTree(Mesh(v=base_verts, f=base_faces))
    nearest_tri, nearest_part, nearest_point = tree.nearest(vert, nearest_part=True)
    nearest_tri = nearest_tri.ravel().astype(np.long)
    nearest_part = nearest_part.ravel().astype(np.long)

    nearest_normals = np.zeros_like(vert)

    #nearest_part tells you whether the closest point in triangle abc is in the interior (0), on an edge (ab:1,bc:2,ca:3), or a vertex (a:4,b:5,c:6)
    cl_tri_idxs = np.nonzero(nearest_part == 0)[0].astype(np.int)
    cl_vrt_idxs = np.nonzero(nearest_part > 3)[0].astype(np.int)
    cl_edg_idxs = np.nonzero((nearest_part <= 3) & (nearest_part > 0))[0].astype(np.int)

    nt = nearest_tri[cl_tri_idxs]
    nearest_normals[cl_tri_idxs] = fn[nt]

    nt = nearest_tri[cl_vrt_idxs]
    npp = nearest_part[cl_vrt_idxs] - 4
    nearest_normals[cl_vrt_idxs] = vn[base_faces[nt, npp]]

    nt = nearest_tri[cl_edg_idxs]
    npp = nearest_part[cl_edg_idxs] - 1
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]
    npp = np.mod(nearest_part[cl_edg_idxs], 3)
    nearest_normals[cl_edg_idxs] += vn[base_faces[nt, npp]]

    nearest_normals = nearest_normals / (np.linalg.norm(nearest_normals, axis=-1, keepdims=True) + 1.e-10)

    return nearest_point, nearest_normals


def remove_interpenetration_fast(mesh, base, L=None):
    """
    Laplacian can be cached.
    Computing laplacian takes 60% of the total runtime.
    """
    import scipy.sparse as sp
    from scipy.sparse import vstack, csr_matrix
    from scipy.sparse.linalg import spsolve
    from psbody.mesh import Mesh

    eps = 0.01
    ww = 2.0
    nverts = mesh.v.shape[0]

    if L is None:
        L = laplacian(mesh)

    nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, base.v, base.f)
    direction = np.sign( np.sum((mesh.v - nearest_points) * nearest_normals, axis=-1) )

    indices = np.where(direction < 0)[0]

    pentgt_points = nearest_points[indices] - mesh.v[indices]
    pentgt_points = nearest_points[indices] \
                    + eps * pentgt_points / np.expand_dims(0.0001 + np.linalg.norm(pentgt_points, axis=1), 1)
    tgt_points = mesh.v.copy()
    tgt_points[indices] = ww * pentgt_points

    rc = np.arange(nverts)
    data = np.ones(nverts)
    data[indices] *= ww
    I = csr_matrix((data, (rc, rc)), shape=(nverts, nverts))

    A = vstack([L, I])
    b = np.vstack((
        L.dot(mesh.v),
        tgt_points
    ))

    res = spsolve(A.T.dot(A), A.T.dot(b))
    mres = Mesh(v=res, f=mesh.f)
    return mres



In [ ]:
import time
#start_time = time.time()
body = Mesh(filename='/content/drive/MyDrive/body_0000.ply')
mesh = Mesh(filename='/content/drive/MyDrive/pred_gar_0000.ply')
start_time = time.time()
mesh1 = remove_interpenetration_fast(mesh, body)
#mesh1.write_ply('/gar.ply')
end_time = time.time()
print(end_time - start_time)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

0.964881181716919


In [ ]:
eps = 0.001
ww = 2.0
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)
direction = np.sign( np.sum((mesh.v - nearest_points) * nearest_normals, axis=-1) )

indices = np.where(direction < 0)[0]

pentgt_points = nearest_points[indices] - mesh.v[indices]
pentgt_points = nearest_points[indices] \
                    + eps * pentgt_points / np.expand_dims(0.0001 + np.linalg.norm(pentgt_points, axis=1), 1)
tgt_points = mesh.v.copy()
tgt_points[indices] = ww * pentgt_points


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
indices.shape

(40,)

In [ ]:
indices

array([  59,   77,  144,  145,  147,  148, 1362, 1365, 1922, 2656, 2657,
       2665, 2666, 2674, 2675, 2750, 2756, 2757, 2761, 2762, 2767, 3467,
       3468, 3472, 3531, 3532, 3533, 3534, 3540, 4720, 4753, 5992, 6075,
       6079, 6080, 6081, 6083, 6084, 6086, 6090])

## Naive nearest point

In [ ]:
import time
#start_time = time.time()
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/1.obj')
nearest_indices = np.load('/content/nearest_indices.npy')
#for i in range()
base_verts = body.v
base_faces = body.f
nearest_points_naive = np.asarray(mesh.v)
nearest_normals_naive = np.asarray(mesh.v)
vn = VertNormals(v=body.v, f=body.f).reshape((-1, 3))
for i in range(6716):
      nearest_points_naive[i] = (body.v[nearest_indices[i]])
      nearest_normals_naive[i] = (vn[nearest_indices[i]])
    
direction = np.sign( np.sum((mesh.v - nearest_points_naive) * nearest_normals_naive, axis=-1) )


In [ ]:
import time
#start_time = time.time()
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/1.obj')
nearest_indices = np.load('/content/near_indx.npy')
#for i in range()
base_verts = body.v
base_faces = body.f
nearest_points_naive = np.asarray(mesh.v)
nearest_normals_naive = np.asarray(mesh.v)
vn = VertNormals(v=body_verts, f=body_faces).reshape((-1, 3))
for i in range(6716):
      nearest_point = body.v[nearest_indices[i]]
      nearest_normal = vn
    
direction = np.sign( np.sum((mesh.v - nearest_points_naive) * nearest_normals_naive, axis=-1) )


In [ ]:
indices = np.where(direction < 0)[0]
indices

array([], dtype=int64)

In [ ]:
print(nearest_points_naive[0])
print(nearest_points[0])


[-0.09629597 -0.15054663  0.98390182]
[1. 1. 1.]


In [ ]:
body.v[nearest_indices[0]]

array([ 0.08219133,  0.40182057, -0.02062367])

In [ ]:
from scipy.spatial import KDTree
body = Mesh(filename='/content/drive/MyDrive/1.obj')
distances = [0]*6716
indices = [0]*6716
import time
start_time = time.time()


kdtree = KDTree(body.v)
print(time.time() - start_time)
mesh_cloth = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
vertices = mesh_cloth.v
# distances, indices = kdtree.query(vertices)
# print(time.time() - start_time)
for i in range(6716):
  distances[i], indices[i] = kdtree.query(vertices[i])
print(time.time() - start_time)

0.003774404525756836
0.19837570190429688


In [ ]:
np.save('/content/near_indx.npy', indices)

In [ ]:
near_idx = np.load('/content/near_indx.npy')

In [ ]:
body.v[near_idx[0]]

array([-0.222243, -3.218895,  1.461225])

In [ ]:
near_idx[0]

8233

In [ ]:
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)

[   0    1    3 ... 6713 6714 6715]
(5571,)
[ 335  376  441  791  823  838 1045 1892 2869 2950 3155 3157 3159 3206
 3772 4049 4267 4419 4424 4437 4574 4580 4583 4834 4922 5560 6317 6448
 6461 6469 6514 6515 6537 6538 6641 6658]
(36,)
[   2   20   32 ... 6709 6710 6711]
(1109,)
[ 8726  8722  8729 ... 13664 18163 13793]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
body = Mesh(filename='/content/drive/MyDrive/1.obj')
mesh = Mesh(filename='/content/drive/MyDrive/predicted_exp.obj')
nearest_points, nearest_normals = get_nearest_points_and_normals(mesh.v, body.v, body.f)

[   0    1    3 ... 6713 6714 6715]
(5571,)
[ 335  376  441  791  823  838 1045 1892 2869 2950 3155 3157 3159 3206
 3772 4049 4267 4419 4424 4437 4574 4580 4583 4834 4922 5560 6317 6448
 6461 6469 6514 6515 6537 6538 6641 6658]
(36,)
[   2   20   32 ... 6709 6710 6711]
(1109,)
[ 8726  8722  8729 ... 13664 18163 13793]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

In [ ]:
nearest_points[0]

array([-0.15373073, -3.28210668,  1.45027695])

In [ ]:
body.v[near_idx[0]]

array([-0.222243, -3.218895,  1.461225])

In [ ]:
for i in range(6716):
    if (body.v[i][0] == nearest_points[0][0]) and (body.v[i][1] == nearest_points[0][1]) and (body.v[i][2] == nearest_points[0][2]):
      print(i) 

In [ ]:
(mesh.v[59][0] - body.v[near_idx[59]][0])*(vn[near_idx[59]][0]) + (mesh.v[59][1] - body.v[near_idx[59]][1])*(vn[near_idx[59][1]]) + (mesh.v[59][2] - body.v[near_idx[59]][2])*(vn[near_idx[59][2]])

IndexError: ignored

In [ ]:
mesh.v[59]

array([-0.54776615,  0.83221754, -0.0858266 ])

In [ ]:
nearest_points[59]

array([-0.87467397,  3.10795966, -0.70369196])

In [ ]:
nearest_points[59]

array([-0.87467397,  3.10795966, -0.70369196])

In [ ]:
nearest_normals[59]

array([-0.72334506,  0.67183807,  0.15939115])

In [ ]:
-0.54 +0.87

0.32999999999999996

In [ ]:
0.83 - 3.10


-2.27

In [ ]:
0.7 - 0.085

0.615

In [ ]:
0.33 * 0.71

0.2343

In [ ]:
body.v[near_idx[59]]

array([-0.903855,  3.08307 , -0.73121 ])

In [ ]:
vn[near_idx[59]]

array([-0.54776615,  0.83221754, -0.0858266 ])